In [1]:
class TrainDataset():
    def __init__(self, path):

        eddies_train = xr.open_dataset(path + '/eddies_train.nc')
        X_train = xr.open_dataset(path + '/OSSE_U_V_SLA_SST_train.nc')
        
        y = eddies_train.eddies.values
        
        X_verti = X_train.vomecrtyT.values
        X_hori = X_train.vozocrtxT.values
        X_SSH = X_train.sossheig.values
        X_SST = X_train.votemper.values

        X_verti = np.nan_to_num(X_verti, nan=np.nanmean(X_verti))
        X_hori = np.nan_to_num(X_hori, nan=np.nanmean(X_hori))
        X_SSH = np.nan_to_num(X_SSH, nan=np.nanmean(X_SSH))
        X_SST = np.nan_to_num(X_SST, nan=np.nanmean(X_SST))
        
        ##Normalisation
        X_verti = ( X_verti - np.min(X_verti) )/( np.max(X_verti) - np.min(X_verti) )
        X_hori = ( X_hori - np.min(X_hori) )/( np.max(X_hori) - np.min(X_hori) )
        X_SSH = ( X_SSH - np.min(X_SSH) )/( np.max(X_SSH) - np.min(X_SSH) )
        X_SST = ( X_SST - np.min(X_SST) )/( np.max(X_SST) - np.min(X_SST) )
        
        
        ##Transformation
        X = np.array([X_verti, X_hori, X_SSH, X_SST])
        X = X.transpose((1,0,2,3))
        y = np.nan_to_num(y, nan=3)
        
        
        self.X_train = torch.tensor(X)
        self.y_train = torch.tensor(y, dtype=torch.long)
        self.image_transforms = image_transforms
        
        
        
    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        
        x = self.X_train[idx]
        y = self.y_train[idx]
        
        x = self.image_transforms(x)
        
        return self.X_train[idx], self.y_train[idx]

In [2]:
from torchvision import datasets, transforms

image_transforms = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomVerticalFlip(0.5),
    ]
)
